<a href="https://colab.research.google.com/github/teobenko99/PRACTICA/blob/main/04_MODELADO_Parte2_TP_Final_AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carga de Librerias

In [ ]:
!pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import recall_score, f1_score, make_scorer

# Load the cleaned dataset
df_final = pd.read_csv("dataset_transformado_ok.csv")

# Universidad Nacional de la Matanza

Especialización en Ciencia de Datos

Materia: APRENDIZAJE AUTOMÁTICO

GRUPO : Benko Teo, Cura Diego, Riganti Valentina, Sanjuan Oriana.

# **Etapa: Modelado - Parte 2**
*Aplicación de Automated Machile Learning (AutoML).*

*Objetivo*: Comparar los resultados obtenidos al aplicar una nueva técnica de AutoML con lo desarrollado usando PyCaret.

In [ ]:
# --- 1. PREPARACIÓN DE DATOS Y SPLIT ---
TOP_15_FEATURES = [
    'VIV_DESCONOCIDO', 'psicologica', 'BA_DESCONOCIDO', 'NE_SECUNDARIO', 'SL_INFORMAL',
    'convivencia_pea_bin', 'SL_DESCONOCIDO', 'RV_parientes_convivientes', 'tratamiento_bin',
    'diagnostico_bin', 'SUMA_VIOLENCIA', 'fisica', 'RV_desconocido', 'PER_desconocido',
    'NE_OTROS'
]
target_col = 'denuncio_target'

# Preparación final del DataFrame
for col in df_final.columns:
    if df_final[col].dtype == 'bool':
        df_final[col] = df_final[col].astype(int)

X = df_final[TOP_15_FEATURES].fillna(0).values
Y = df_final[target_col].values

# División de datos (80/20 estratificada)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42, stratify=Y
)

# --- 2. OPTIMIZACIÓN CON GRIDSEARCHCV (Simulando AutoML) ---

# 2.1. Definir el modelo
lgbm = LGBMClassifier(random_state=42, class_weight='balanced')

# 2.2. Definir el espacio de búsqueda (Hiperparámetros)
param_grid = {
    'n_estimators': [50, 100],        # Número de árboles
    'learning_rate': [0.05, 0.1],     # Tasa de aprendizaje
    'num_leaves': [10, 20],           # Complejidad del árbol
}

# 2.3. Definir el Scorer: Creamos un objeto de métrica que optimice el RECALL
recall_scorer = make_scorer(recall_score)

# 2.4. Inicializar GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring=recall_scorer,
    cv=5, # 5-fold Cross-Validation
    n_jobs=-1,
    verbose=2
)

print("--- Iniciando GridSearchCV (Simulando Tuning AutoML) ---")
grid_search.fit(X_train, Y_train)

# --- 3. EVALUACIÓN FINAL ---

# 3.1. Obtener el mejor modelo
best_lgbm = grid_search.best_estimator_

# 3.2. Predicción en el conjunto de prueba
Y_pred = best_lgbm.predict(X_test)

# 3.3. Obtener Métricas
final_recall = recall_score(Y_test, Y_pred)
final_f1 = f1_score(Y_test, Y_pred)

print("\n--- Resultados Finales de LightGBM (Optimizado) ---")
print(f"Mejor Parámetro Encontrado: {grid_search.best_params_}")
print(f"Recall (Clase 1, Alerta): {final_recall:.4f}")
print(f"F1-Score (Balance): {final_f1:.4f}")

--- Iniciando GridSearchCV (Simulando Tuning AutoML) ---
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Info] Number of positive: 76, number of negative: 112
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30
[LightGBM] [Info] Number of data points in the train set: 188, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


# Resultados

El LightGBM Classifier, aunque es un modelo potente basado en árboles, no logró superar el rendimiento del modelo lineal (Ridge) en la métrica de alerta crucial (Recall).

* Recall (Clase 1) --> 0.5263 (52.6%) vs 0.6842 (68.4%)
RENDIMIENTO INFERIOR. El LightGBM omite más Falsos Negativos.

F1-Score --> 0.5263 vs 0.6500
RENDIMIENTO INFERIOR. Peor balance entre sensibilidad y precisión.

* Mejor Parámetro: El mejor modelo es relativamente simple (no muy profundo ni con muchos árboles).

{'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 10}

**Decisión Final**

---


El Modelo Seleccionado es Ridge Classifier. A pesar de ser más simple, su naturaleza lineal fue más efectiva para detectar los patrones de "falla de registro" (como VIV_DESCONOCIDO) que son altamente correlacionados y lineales. El LightGBM, al buscar interacciones complejas, no pudo superar el peso de estas características.

Por otro lado, un Recall del $\mathbf{0.5263}$ es insuficiente para un sistema de alerta, ya que significaría omitir casi la mitad ($\approx 47\%$) de los casos de riesgo real. El Recall de $\mathbf{0.6842}$ del Ridge Classifier es superior.

Finalizando, se puede decir que aunque se probaron modelos no lineales avanzados (LightGBM) que deberían ser superiores, el Ridge Classifier demostró ser el más robusto y el que ofrece el mejor equilibrio (Recall/F1-Score), lo cual es esencial para la implementabilidad del sistema.